In [6]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import preprocessing
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Load test scores csv
ts = pd.read_csv('test_scores.csv')

In [7]:
ts.describe()

,n_student,pretest,posttest
count,2133.000000,2133.000000,2133.000000
mean,22.796531,54.955931,67.102203
std,4.228893,13.563101,13.986789
min,14.000000,22.000000,32.000000
25%,20.000000,44.000000,56.000000
50%,22.000000,56.000000,68.000000
75%,27.000000,65.000000,77.000000
max,31.000000,93.000000,100.000000


In [34]:
# # replace school_setting values to Integers
# ts['school_setting'] = ts['school_setting'].map({'Urban': 1, 'Suburban': 2, 'Rural': 3})

# # replace school_type values to Integers
# ts['school_type'] = ts['school_type'].map({'Public': 1, 'Non-public': 0})

# # replace teaching_method values to Integers
# ts['teaching_method'] = ts['teaching_method'].map({'Standard': 1, 'Experimental': 0})

# # replace gender values to Integers
# ts['gender'] = ts['gender'].map({'Male': 1, 'Female': 0})

# # replace lunch values to Integers
# ts['lunch'] = ts['lunch'].map({'Does not qualify': 1, 'Qualifies for reduced/free lunch': 0})
# ts



ts['school_setting'] = ts['school_setting'].replace("Urban", 1)
ts['school_setting'] = ts['school_setting'].replace("Suburban", 2)
ts['school_setting'] = ts['school_setting'].replace("Rural", 3)

# replace school_type values to Integers
ts['school_type'] = ts['school_type'].replace("Public", 1)
ts['school_type'] = ts['school_type'].replace("Non-public", 0)

# replace teaching_method values to Integers
ts['teaching_method'] = ts['teaching_method'].replace("Standard", 1)
ts['teaching_method'] = ts['teaching_method'].replace("Experimental", 0)

# replace gender values to Integers
ts['gender'] = ts['gender'].replace("Female", 1)
ts['gender'] = ts['gender'].replace("Male", 0)

# replace lunch values to Integers
ts['lunch'] = ts['lunch'].replace("Qualifies for reduced/free lunch", 1)
ts['lunch'] = ts['lunch'].replace("Does not qualify", 0)
ts

x2 = ts.drop(['classroom', 'student_id','posttest'], axis=1)
y2 = ts['posttest']

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categories = ['school', 'school_setting', 'school_type', 'teaching_method', 'gender', 'lunch']

one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot', one_hot, categories)],
                                remainder = 'passthrough')


x_transformed = transformer.fit_transform(x2)

<1706x36 sparse matrix of type '<class 'numpy.float64'>'
	with 13648 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def evaluate(y_act,y_pred):
    MAE = mean_absolute_error(y_act, y_pred)
    MSE = mean_squared_error(y_act, y_pred)
    r2 = r2_score(y_act, y_pred)
    eval_values = {'MAE': MAE, 'MSE' : MSE, 'r2' : r2}
    print('Results:\n Mean absolute error = ', round(MAE,3),'\n Mean squared error = ', round(MSE,3),'\n R2 = ', round(r2,3))

    return eval_values


In [22]:
from sklearn.model_selection import train_test_split

x = ts.drop(columns=['posttest', 'school', 'classroom', 'student_id'])
y = ts['posttest']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.3, random_state=17)

x_train

,school_setting,school_type,teaching_method,gender,lunch,n_student,pretest
1231,3,1,1,0,0,23,59
2013,2,1,0,0,0,23,74
1435,2,1,1,0,0,21,58
1959,2,1,0,0,0,24,65
1737,3,1,1,1,0,18,47
...,...,...,...,...,...,...,...
278,1,1,1,1,0,27,50
1965,2,1,0,0,1,24,65
1337,1,1,0,0,1,25,62
406,2,1,1,1,1,20,64


In [23]:
from sklearn.preprocessing import StandardScaler

# Feature scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)

In [24]:
from sklearn.linear_model import RidgeCV

ridge_cv = RidgeCV()
ridge_cv.fit(x_train_scaled, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [27]:

ridge_cv_predict = ridge_cv.predict(x_valid_scaled)

# Learning Assessment
scores = evaluate(y_valid, ridge_cv_predict)
# print('MAE:', mean_absolute_error(y_valid, ridge_cv_predict))

Results:
 Mean absolute error =  2.528 
 Mean squared error =  10.305 
 R2 =  0.949


In [25]:
model = DecisionTreeClassifier()
model.fit(x,y)

DecisionTreeClassifier()

In [26]:
# Male vs. Female
model.predict([[1,1,1,1,1,20], [1,1,1,1,1,20]])

ValueError: X has 6 features, but DecisionTreeClassifier is expecting 7 features as input.

In [23]:
#
model.predict([[1,1,1,1,1,20], [1,1,1,1,1,20]])

array([61, 61])